<a href="https://colab.research.google.com/github/eumarcosdeoliveira/Os-Amostradinhos/blob/Atividades/sparkAilton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdulszz/spotify-most-streamed-songs")

print("Path to dataset files:", path)

c:\Users\guilh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 60.0k/60.0k [00:00<00:00, 480kB/s]

Extracting files...
Path to dataset files: C:\Users\guilh\.cache\kagglehub\datasets\abdulszz\spotify-most-streamed-songs\versions\2


In [ ]:
 # Iniciar uma Sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploPySpark") \
    .getOrCreate()

In [ ]:
import pandas as pd
df_pandas = pd.read_csv("Spotify Most Streamed Songs.csv")


In [ ]:
df_pandas.head(4)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,cover_url
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,B,Major,80,89,83,31,0,8,4,Not Found
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,C#,Major,71,61,74,7,0,10,4,https://i.scdn.co/image/ab67616d0000b2730656d5...
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,F,Major,51,32,53,17,0,31,6,https://i.scdn.co/image/ab67616d0000b273e85259...
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,A,Major,55,58,72,11,0,11,15,https://i.scdn.co/image/ab67616d0000b273e787cf...


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ExemploLeitura").getOrCreate()

# Leitura de um arquivo CSV"
df_pyspark = spark.read.csv("Spotify Most Streamed Songs.csv", header=True, inferSchema=True)

In [ ]:
 #Nomes das colunas
for i in df_pandas.columns:
  print(i)

track_name
artist(s)_name
artist_count
released_year
released_month
released_day
in_spotify_playlists
in_spotify_charts
streams
in_apple_playlists
in_apple_charts
in_deezer_playlists
in_deezer_charts
in_shazam_charts
bpm
key
mode
danceability_%
valence_%
energy_%
acousticness_%
instrumentalness_%
liveness_%
speechiness_%
cover_url


In [ ]:
# Seleciona colunas específicas - Pandas
df_selecionado_pandas = df_pandas[['artist_count', 'released_year', 'streams']]

In [ ]:
# Seleciona colunas específicas - PySpark
df_selecionado_spark = df_pyspark.select('artist_count', 'released_year', 'streams')

In [ ]:
df_filtrado_pandas = df_pandas[df_pandas['artist_count'] > 2]


In [ ]:
df_filtrado_spark = df_pyspark.filter(df_pyspark.artist_count > 2)

In [ ]:
df_agregado_pandas = df_pandas.groupby('artist(s)_name')['artist_count'].mean().reset_index()

In [ ]:
# Calcula a média de coluna1 agrupada por coluna2 - PySpark
from pyspark.sql.functions import avg

# Calcula a média de coluna1 agrupada por coluna2
df_agregado_spark = df_pyspark.groupBy("in_spotify_playlists").agg(avg("streams").alias("media_streams"))

In [ ]:
df_selecionado_spark.show(5)
df_filtrado_spark.show(5)
df_agregado_spark.show(5)

+------------+-------------+---------+
|artist_count|released_year|  streams|
+------------+-------------+---------+
|           2|         2023|141381703|
|           1|         2023|133716286|
|           1|         2023|140003974|
|           1|         2019|800840817|
|           1|         2023|303236322|
+------------+-------------+---------+
only showing top 5 rows

+--------------------+--------------------+------------+-------------+--------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+----------------+---+----+-----+--------------+---------+--------+--------------+------------------+----------+-------------+--------------------+
|          track_name|      artist(s)_name|artist_count|released_year|released_month|released_day|in_spotify_playlists|in_spotify_charts|   streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|bpm| key| mode|dance

In [ ]:
# Junção de dois DataFrames pandas
df1 = pd.read_csv("Companies_ranked_by_Dividend_Yield.csv")
df2 = pd.read_csv("Companies_ranked_by_Earnings.csv")
df_juncao = pd.merge(df1, df2, on="Rank", how="left", suffixes=('_dividend', '_earnings'))


In [ ]:
# Junção de dois DataFrames PySpark
df1 = spark.read.csv("Companies_ranked_by_Dividend_Yield.csv", header=True, inferSchema=True)
df2 = spark.read.csv("Companies_ranked_by_Earnings.csv", header=True, inferSchema=True)
df_juncao = df1.join(df2, on="Rank", how="left")